# Notes for Computational Fluid Dynamics
The Basics with Applications by John D. Anderson Jr.

Chapter 2 (cont). and Chapter 3 and Chapter 4 and Chapter 5

In [27]:
import numpy as np
import scipy as sp
import sympy as sy
from matplotlib import pyplot as plt

## Chapter 2

### Chapter 2.10 Considerations for CFD

Conservation forms can be written as:

$$\frac{\partial U}{\partial t} + \frac{\partial F}{\partial x} + \frac{\partial G}{\partial y} + \frac{\partial H}{\partial z} = J$$

Notes to self:
- Each captial letter represents a column vector
- $\rho$ = density
- $p$ = pressure
- $u, v, w$ are the velocity of the fluids in (x, y, z)
- $e$ is the internal energy
- $V^{2}$ is the squared magnitude of the vel vector ($u^{2} + v^{2} + w^{2}$)
- $\tau_{ij}$ is the normal or sheer forces exerted by velocity differentials in a viscous fluid on a plane perpendicular to i and the force is exerted in the j direction)
- $k$ and $T$ are temperature-energy components
- $\dot{q}$ is the heat influx into the unit
- $f_{k}$ is the body force (E.g. gravity) in the $k$ direction

Back to the maths

#### Viscous Flow/Navier-Stokes

$$U =
\begin{bmatrix}
\rho \\
\rho u \\
\rho v \\
\rho w \\
\rho \left( e + \frac{V^{2}}{2} \right)
\end{bmatrix}$$

$$F =
\begin{bmatrix}
\rho u \\
\rho u^{2} + p - \tau_{xx} \\
\rho vu - \tau_{xy} \\
\rho wu - \tau_{xz} \\
\rho \left( e + \frac{V^{2}}{2} \right)u + pu - k \frac{\partial T}{\partial x} - u \tau_{xx} - v \tau_{xy} - w \tau_{xz}
\end{bmatrix}$$

$$G =
\begin{bmatrix}
\rho v \\
\rho uv + \tau_{yx} \\
\rho v^{2} + p - \tau_{yy} \\
\rho wv - \tau_{yz} \\
\rho \left( e + \frac{V^{2}}{2} \right)v + pv - k \frac{\partial T}{\partial y} - u \tau_{yx} - v \tau_{yy} - w \tau_{yz}
\end{bmatrix}$$

$$H =
\begin{bmatrix}
\rho w \\
\rho uw - \tau_{zx} \\
\rho vw - \tau_{zy} \\
\rho w^{2} + p - \tau_{zz} \\
\rho \left( e + \frac{V^{2}}{2} \right)w + pw - k \frac{\partial T}{\partial z} - u \tau_{zx} - v \tau_{zy} - w \tau_{zz}
\end{bmatrix}$$

$$J =
\begin{bmatrix}
0 \\
\rho f_{x} \\
\rho f_{y} \\
\rho f_{z} \\
\rho (uf_{x} + vf_{y} + wf_{z}) + p \dot{q}
\end{bmatrix}$$

Where U is the solution vector for computed variables. "Primitive" variables can be extracted from the computed "flux" variables:

$$\rho = \rho$$

$$u = \rho u / \rho$$

$$v = \rho v / \rho$$

$$w = \rho w / \rho$$

$$e = \frac{\rho \left( e + \frac{V^{2}}{2} \right)}{\rho} - \frac{u^{2} + v^{2} + w^{2}}{2}$$

In [28]:
'''
This note from the text seems to suggest a convinient Python transfer
where each object has underlying numpy matrices from which Python
properties are calculated for the primitive values that the output
must consider (e.g. density, pressure) which can be defined for both
their single variable name (e.g. rho, p) and their associated full name
(e.g. density, pressure respectively)

A Python object approach could allow for "easy" conversion of the object
into a tri or quad for visualization in a heatmap for each property, or 
probably more simply, a characteristic point in approximately the center
of the polygon can be selected and drawn with some radius + colored to
make an approximate heatmap, with more points draw where the polygons
are dense
'''

# Not sure on this representation yet
class HyperPlane(object):
    pass

# This could be a named tuple right now
class Point(object):
    def __init__(self):
        self._x = 0.0
        self._y = 0.0
        self._z = 0.0

# TODO(buckbaskin): Make these comments docstrings, so help(...) is
#   useful for programmers and CFD
class Region(object):
    def __init__(self):
        self.neighbors = {}  # Dict[HyperPlane, Region]
        self.characteristic_point = Point()
        self._U = np.zeros((1, 5))
        self._F = np.zeros((1, 5))
        self._G = np.zeros((1, 5))
        self._H = np.zeros((1, 5))
        self._J = np.zeros((1, 5))
        self.parent = None
        self.children = []
    
    def rho(self):
        # probably a property
        # ooh, shiny, self documenting code ;)
        return self.density()

    def density(self):
        # TODO(buckbaskin): make this a read property
        return self._U[0, 0]
    
    def pressure(self):
        # This is more complicated
        raise NotImplementedError()
    
    def volume(self):
        # return the "volume" contained in the hyperplanes
        pass
    
    def u(self):
        # x velocity
        pass
    
    def v(self):
        # y velocity
        pass
    
    def w(self):
        # potential z velocity, 0.0 for now
        return 0.0
    
    def velocity(self):
        return np.array((self.u, self.v, self.w,))
    
    def V(self):
        # probably a property
        # ooh, shiny, self documenting code ;)
        return velocity()

# values for this class are fixed, but take on the same values as a
#   Region. Is there a nice Pythonic way to make this class read-only
#   after creation? Pass a weakref? Ask people to play nicely, so they
#   can have time-varying boundary conditions if they want?
class BoundaryRegion(Region):
    pass

# something else will build a network of regions
# something else will update regions for each time step based on their
#   neighbors and keep track of which is updated and so-forth
r = Region()

#### Invisicid Flow (Euler)

Note lack of Temperature considerations and lack of internal stresses (normal and sheer $\tau$).

$$U =
\begin{bmatrix}
\rho \\
\rho u \\
\rho v \\
\rho w \\
\rho \left( e + \frac{V^{2}}{2} \right)
\end{bmatrix}$$

$$F =
\begin{bmatrix}
\rho u \\
\rho u^{2} + p \\
\rho vu \\
\rho wu \\
\rho \left( e + \frac{V^{2}}{2} \right)u + pu
\end{bmatrix}$$

$$G =
\begin{bmatrix}
\rho v \\
\rho uv \\
\rho v^{2} + p\\
\rho wv \\
\rho \left( e + \frac{V^{2}}{2} \right)v + pv
\end{bmatrix}$$

$$H =
\begin{bmatrix}
\rho w \\
\rho uw \\
\rho vw \\
\rho w^{2} + p \\
\rho \left( e + \frac{V^{2}}{2} \right)w + pw
\end{bmatrix}$$

$$J =
\begin{bmatrix}
0 \\
\rho f_{x} \\
\rho f_{y} \\
\rho f_{z} \\
\rho (uf_{x} + vf_{y} + wf_{z}) + p \dot{q}
\end{bmatrix}$$

Note to self:
- When body forces are 0 / small and heat flux is 0 / small, the J term ("source" term) is 0

The text recommends isolating $U$ in the form:

$$\frac{\partial U}{\partial t} = J - \frac{\partial F}{\partial x} - \frac{\partial G}{\partial y} - \frac{\partial H}{\partial z}$$

#### Re: shock capturing vs fitting

The hyperplane solution suggests that one can design the polygon/volume splitting algorithm such that one automatically attempts to "fit" shock waves or other events by splitting and achieving hyper-resolution in pertinent areas.

**Note to self**:
Where did rotational energy/angular velocity go in the flow? For small volumes of flow all should be moving in the same direction, I'm not sure this holds for finite "large" fixed volumes.

## Chapter 3

## Chapter 4

Using the column vector approach above, one circumvents the need for higher order derivatives.

### 4.1, 4.2 Numerical quotients

First and second order calculations from Taylor series or polynomial approximations

### 4.3 Difference Equations

For an equation involving multiple derivatives/independent variables, pick a starting condition (at time zero) and insert your approximations of the variables and derivatives of such. This gives a step w.r.t. time (time derivative of the output). Choose some small step forward in time, and update the values based on the estimated derivative. Re-calculate your state variables and repeat.

### 4.4 Explicit :) vs. Implicit Solutions

Explicit form means that you end up building 1 equation at each simulation point (finite point method as opposed to finite volume) and know the conditions at the surrounding points to create a direct/explicit calculation for the values at the current point at the next time step.

- The pro for explicit methods is that they're easy to set up
- The con for explicit methods is that they require small $\Delta t$ and small $\Delta x$ components to remain stable. This means that I'll have to subdivide more to get to spaces where my assumptions hold enough to be stable.
- The pro: explicit methods enable massive parallelization

Note to self:
- In my case, I intend to calculate the updates over time for the next time step of each volume, given the conditions in each volume and assuming a consistent property across each volume, with an instantaneous change in properties across the edge of each volume. My goal would be to iterate over time from my initial and boundary conditions until I reach an approximately steady state, which becomes my marched-to steady state solution for the flow around that hull shape.

Implicit methods involve writing a solution involving other properties at the current time and solving a system of equations at the next time step. This allows for the use of multi-sided derivative methods, but I'd prefer not to have to repeatedly solve the same system of equations.

- The con: Hard to program/set up
- The pro: stable for longer $\Delta t$, resulting in overall less computation time, even though (con) each step requires a large matrix multiplication

### 4.5 Errors and stability

Note to self:
- Ooh, shiny, I could have the nodes estimate their error bounds as well as track how much their key properties have changed over time (difference between slow and fast moving average?) 

This chapter discusses how to calculate numerical error and identify how it propagates through the update step (easier for explicit ;) ) . If, for a given time step or other variable step, the propagation process amplifies numerical error, the likelihood of instability is high. If the propagation process decreases the effects of numerical error (or I guess there's the unlikely case where it doesn't increase them) the likelihood of instability for that time step is low.

#### 4.5.1 Closing thoughts on stability

The coverage of stability here is minimal, the goal is only that you remember that it matters.

### 4.6 Ch. 4 Summary

Discretization matters. Problem 4.7 dicretizes an integral solution (probably of interest to me). CFD techniques are a combination of tools presented in this and preceding chapters (see notes to self about the tools I think I'll use). Chapter 6 discusses techniques and Part 3 goes into detail applying some of these techniques to classic CFD problems.

#### Guidepost
- Option 1: Skip to implementing something, go to Sections 9.1 to 9.3 (Couette incompressible viscous flow, implicit solution) and Problem 9.1 (Couette incompressible viscous flow, explicit solution). This focuses on comparing implicit and explicit solutions.
- Option 2: Skip to implementing a longer something. Go to section 6.3 MacCormack's technique, then go to Sections 7.1 to 7.4, the computer solution of an explicit, time-marching solution to Euler equations for quasi-one dimensional nozzle flow. The important aspects focused here are time-marching finite-difference solutions, partially involving artificial viscosity.
I think I'm going to keep reading for now.

The guidepost skips Chapters 5 and 6, which discusses grid generation and transformations.

## Chapter 5 Grids and Transforms

The proper grid can make or break the numerical solution.

### 5.1 Introduction

Non-uniform grids for finite difference methods are almost always dealt with on a uniform grid or a non-uniform grid that can be warped by transformation into some uniform grid in another representation (e.g. polar) then transforming the underlying equations into that space (e.g. consider velocity in polar). These transform requirements are mitigated by using a non-finite-difference method.

Considerations for a Grid:
1. Some points fall inside the airfoil, what properties do you assign there?
    - In the hyperplane solution, no slice is made inside the airfoil
2. Some small number of grid points may fall on the airfoil (hull). This isn't good, because the properties along the surface are important and the airfoil should be seen as a continuous solid by the numerical solution.
    - In the hyperplane solution, slices along the airfoil are assigned continuous solid properties that prevent intrusion by the fluid and calculation along the surface (with higher resolution with more sub-division).

In [29]:
class SurfaceRegion(Region):
    '''
    Assign values or booleans in this region to aid calulation of flows
    near the surface that this represents. This is a special case of
    boundary regions I think, but I don't want to force it right now.
    '''
    pass

### 5.1 (cont.)

They propose a curve-linear grid where the airfoil shape is a constant $\eta$, and then the airfoil is expanded at integer distance from the surface to create radial steps, with perpendicular lines drawn $\zeta$ at constant "angular" position around the airfoil. From here, define a 1 to 1 correspondence between the x-y world and this $\eta$ - $\zeta$ world. In $\eta$ - $\zeta$ land, the world is uniform and finite differences can be calculated in $\Delta \eta$ and $\Delta \zeta$. Time can also be warped into a new variable $\tau$ as a function of time (note that $\tau$ is solely a function of time). The general mapping is:

- $\zeta = \zeta (x, y, t)$
- $\eta = \eta (x, y, t)$
- $\tau = \tau (t)
This relation can be analytical (nice) or numeric (less nice). Given an analytical definition of the relation, derivatives w.r.t. x, etc. are substituted in the following manner (because chain rule or something):

$$\frac{\partial}{\partial x} = \frac{\partial}{\partial \eta} \frac{\partial \eta}{\partial x} + \frac{\partial}{\partial \zeta} \frac{\partial \zeta}{\partial x} + \frac{\partial }{\partial \tau} \frac{\partial \tau}{\partial x}$$

Note for most variables (including x): $$\frac{\partial \tau}{\partial x} = 0$$

$$\frac{\partial}{\partial t} = \frac{\partial}{\partial \eta} \frac{\partial \eta}{\partial t} + \frac{\partial}{\partial \zeta} \frac{\partial \zeta}{\partial t} + \frac{\partial }{\partial \tau} \frac{\partial \tau}{\partial t}$$

### 5.6 Stretched/Compressed Grids

This occurs to add extra grid points in "areas of interest" around boundaries with nice properties in cartesian space (or not I guess, but on some nice space like cartesian or polar).

### 5.7 Boundary Fitted Coordinates aka Elliptic Grid Generation

This is the case where we want to fit around a weird shape, maybe an airfoil or hull ;). This is only weird for the recombining point at the end of an airfoil.

### 5.8 Adaptive Grids

Areas of interest are areas with large gradients in the flow field properties. Stretched and elliptic grids attempt to pre-identify areas of interest. This is of particular importance in boundary layers. Adaptive grids place points in the flow field where it observes large gradients during early iterations and removes points where there are not gradients.

In [30]:
'''
Thinking about the adaptive grids ideas

Regions should know how to sub-divide themselves given an intersecting
hyperplane (or return themselves and None for a non-intersecting one).

Regions should know how to combine themselves with a neighbor where
feasible. Feasible might mean where you can get a convex representation.
This is where a tree structure might matter the mostest, because you can
recombine any set of nodes into their common parent, which is a
pre-known region. There's the additional case of where two regions share
some common set of hyperplanes (three, one on each side and a third for
their common border). This last part could be acomplished by some
SOL-JWF tree rotations ala self-balacing trees to merge via the common
parent method (instead of trying to make some fancy algorithm for math-
identifying when you can merge).

I'm now convinced that there should be a tree with internal and leaf
nodes that know all their neighbors (dict of hyperplane to neighbor)
and their children. If they have no children, they are a leaf node and
their properties are directly updated during simulation steps.
Otherwise, their approximate properties are given as a summary gathering
of their child nodes.

This also suggests that it would be helpful for the bulk data structure
to track both the root and the leaves of the tree so that one does not
have to iterate through the non-leaf nodes of tree every time. If one
only iterates through leaf nodes, one can choose to store only a single
leaf node and use neighbors to iterate efficiently through nodes as
opposed to maintaining a separate list.

If the neighbors graph is directed, then I'm making an intuition guess
that the outbound neighbors of a leaf node will only point to leaf
nodes. Prove to self:
Base Case:
- Original node is a leaf with no neighbors.
- Split this node with a hyperplane
- Result is two children that each point to eachother as neighbor and
    the original node as their parent. They only point to leaf nodes.

For some arbitrary leaf node that only points to leaf nodes:
- Split this node with a hyperplane
- Result is two regions, with all prior neighbors falling into 3 sets:
    1. Neighbors bordering both children. These neighbors' hyperplane
        edges were split by the incoming hyperplane. Remove the parent
        reference and point to both left and right.
    2. Neighbors bordering the "left" child.
        They are no longer neighbors with the "right" child, so remove
        the parent reference and only point to the "left" child.
    3. Neighbors bordering the "right" child. Remove the parent
        reference and only point to the "right" child.
Outbound neighbors are still referring to leaf nodes (those haven't
changed) and inbound neighbors are updated to refer to just the other
leaf nodes. This requires extra work during map updates but less wasted
time filtering out non-leaf nodes during simulation steps.
'''

# assert that these should exist
print(r.parent)
print(r.children)

class Space(Region):
    '''
    This region represents the infinite space that bounds all nodes.
    The periphery of this should be BoundaryRegions. I might be able
    to skip this node and instead just construct an initial tree with
    boundary regions if I'm careful.
    '''
    pass

None
[]


### 5.8 Adaptive Grids (cont)

Stationary when the flow reaches a steady state. 

Note to self: This means that gradients in x or y can be ok, but gradients/changes over time are important to subdivide. It probably makes sense to subdivide nodes that change significantly in time along some other dimension. Perhaps choose their greatest gradient or maybe choose most important gradient (velocity? energy? pressure?). Maybe this important gradient comes from an attempt to automatically fit
shock waves to select the important gradient (velocity?). The simple example given picks a primitive variable (pressure).

This is where the numerical definition of grid and derivatives come in handy. Also, this is where the $t$ component of the $\eta$ and $\zeta$ functions come in handy.

My intuition that flow fields don't have cascading effects away from solid objects its solidly wrong based on the adaptive field visualization. The adaptive field view is a nice **visualization**

This recommends reading further in modern adaptive grid generation papers. Maybe I can find a survey for **further reading**? At the time of writing modern grid generation was elliptic boundary CFD with adaptive mesh along the surface and into space. Additionally, multiple regions can be made into isolated zones with their own mapping and a transfer relation at the boundary with other zones.

#### An ooh, shiny thought that might make some people cringe
Track the gradient for each variable in time and all other variables, or provide enough state that it can be calculated at the given time from this step and the previous step as an additional property of the region.

A fancy way: instead of making all the returns just numbers, make them all classes of their own, and define a global gradient function that takes that class and returns its stored gradient. Otherwise, the class is used as you'd expect a number to behave based on its value as opposed to its gradient. The meta version would be that the gradient output itself is a class which can have gradient called on it.

This is probably too fancy for now, but oh, the possibilities. This would be made better by having the "variables" as computed properties of the Region class. I'm starting to love and worry about this rabbit hole.

#### More thoughts

- If I pick the solver that considers viscous forces, and I define my boundaries right and I define my surface properties correctly, I can get the pressure drag ($\delta p$ front to back of the craft times area) and the surface drag of the shape (viscous shear forces).
- If I pick larger regions, I could treat them as zones and then 

### 5.10 Unstructured Meshes

"Constructing an unstructured mesh might be considered a work of art..."

Also, people are just cutting holes into the blocks of a cartesian grid as a cartesian mesh and working from there. Blocks are either open, cut or filled and have a fixed number of neighbors. Cartesian mesh also makes for easy scaling by quad tree, but that makes me unhappy. I want to do better :)